# Tutorial \#3: LibKet - Advanced Features
In this third part of the tutorial you will learn to write quantum algorithms as quantum expressions and to execute them on different quantum computer backends. You will also learn to implement generic quantum expressions as building blocks for larger quantum algorithms.

## Getting started
Let's include **LibKet**'s main headerfile and import its namespaces

In [ ]:
#include "LibKet.hpp"
using namespace LibKet;
using namespace LibKet::circuits;
using namespace LibKet::filters;
using namespace LibKet::gates;

## Quantum expressions

Let us revisit a simple program from the second part of this tutorial and have a look at what `to_string()` returns

In [ ]:
QProgram prog1;
prog1.h(0);
prog1.cnot(0,1);
std::cout << prog1.to_string() << std::endl;

This is a **quantum expression** that is generated from the **quantum program**. 

It needs to be read from inside to outside, i.e. first all qubits are initialized (`init()`) which is the default behavior when creating a new `QProgram` object. 

Next,the zeroth qubit is selected (`sel_<0>(...)`) and a Hadamard gate (`h(...)`) is applied. 

Finally, the outcome of the previous operations is *streamed* through the CNOT gate (`cnot(..., ...)`), whereby the zeroth qubit is selected as control qubit and the first qubit is the one to which the NOT operation is applied to.

We can use the same notation to create **quantum expressions** ourselves

In [ ]:
auto expr1 = cnot(sel_<0>(),sel_<1>(h(sel_<0>(init()))));
std::cout << expr1 << std::endl;

What we see is that **LibKet** provides aliases for all gates, i.e. the Hadamard gate can be accessed with `h()`, `hadamard()`, `H()`, and `HADAMARD()`.

Another way of looking at the quantum expression is to inspect its [abstract syntax tree](https://en.wikipedia.org/wiki/Abstract_syntax_tree) (AST) which we print with

In [ ]:
show(expr1)

This only shows the top-most level of the syntax tree. Here, the CNOT gate. To see the full syntax tree, we increase the level up to which we want to print the AST.

In [ ]:
show<99>(expr1)

## Filters and gates

This way of looking at quantum algorithms might be non-intuitive at first sight but is has some advantages. Each level can be of type `UnaryQGate`, `BinaryQGate`, or `TernaryQGate` and has the attributes `gate`, `filter` and `exprX`.
```text
BinaryQGate
|   gate = QCNOT
| filter = QFilterSelect [ 0 1 ]
|  expr0 = QFilterSelect [ 0 ]
|  expr1 = UnaryQGate
|          |   gate = QHadamard
|          | filter = QFilterSelect [ 1 ]
|          |   expr = UnaryQGate
|          |          |   gate = QInit
|          |          | filter = QFilterSelect [ 0 ]
|          |          |   expr = QFilter
```

Let's start at the innermost level
```text
                      UnaryQGate
|          |          |   gate = QInit
|          |          | filter = QFilterSelect [ 0 ]
|          |          |   expr = QFilter
```
which performs the operation `gate = QInit`. *Afterwards* the `filter = QFilterSelect [ 0 ]` is applied, that is, the zeroth qubit is selected and all upstream operations are performed on this subset of qubits unless the filter is reset.

The next level is
```text
UnaryQGate
|          |   gate = QHadamard
|          | filter = QFilterSelect [ 1 ]
|          |   expr = <INPUT FROM PREVIOUS LEVEL>
```
whereby the `expr` onto which the `gate = QHadamard` operation is applied comes from the previous level. Again, the selection of qubits is modified *afterwards* through `filter = QFilterSelect [ 1 ]`.

The outermost level 
```text
BinaryQGate
|   gate = QCNOT
| filter = QFilterSelect [ 0 1 ]
|  expr0 = QFilterSelect [ 0 ]
|  expr1 = <INPUT FROM PREVIOUS LEVEL>
```
performs the CNOT operation using the input from the previous level as `expr1`, i.e., the zeroth qubit (`QFilterSelect [ 1 ]`) is the one to which the CNOT operation is applied to. The control qubit is specified by `expr0 = QFilterSelect [ 0 ]`.

## More on filters

**LibKet's** filter mechanism can do much more than just selecting individual qubits one by one.

*   `qubit_<4>` selects one individual qubit, i.e. `q4`
*   `sel_<0,1,4,8>` selects individual qubits, i.e. `q0`, `q1`, `q4` and `q8`
*   `range_<0,4>` selects qubit range, i.e. `q0` - `q4`

The trailing `_` indicates that the selection is absolute. That is, no matter which qubits have been selected before the next filter first zooms out to *all* qubits and then selects the specified ones. Hence, `sel_<4>(...)` is equivalent to `sel<4>(all(...))`.

All filters also exist as relative versions (without trailing `_`), whereby the filter is applied relative to the selection of incoming qubits.

In [ ]:
show<99>(range<1,3>(sel_<1,3,8,7,6>(init())));

#### Exercise 1
Write a quantum expression that applies the [first Bell state](https://en.wikipedia.org/wiki/Bell_state) $ \lvert \beta_{00}\rangle = \frac{1}{\sqrt{2}}(\lvert 00 \rangle + \lvert 11 \rangle) $ 

![Bell state](images/multi_qubit_circuit_HI_CNOT.png)

to any two incoming qubits (specified by the expression `expr2_in`) and validate its correct functioning for different qubit selections from a 4-qubit device.

In [ ]:
auto expr2_in = sel_<2,3>(init());

// your code goes here

QDevice<QDeviceType::quest, 4> device2;
device2(expr2);
device2.eval(1);

std::cout << device2.reg() << std::endl;

## Changing the quantum backend

In order to make sure that we start from the same quantum expression let us create the first Bell state again this time with measurement at the end

In [ ]:
auto expr_bell = measure(cnot(h(sel<0>()), sel<1>(init())));

In what follows we will execute this quantum expression on two different quantum backends.

![Cirq](images/cirq_logo.png)

[Cirq](https://quantumai.google/cirq) is a Python package for writing, manipulating, and optimizing quantum circuits developed by Google. It is seamlessly integrated into **LibKet** using the embedded Python interpreter, so you should not notice all the technical details going on under the hood.

In [ ]:
QDevice<QDeviceType::cirq_simulator, 2> cirq;
utils::json result = cirq(expr).eval(1);

Unlike the previously used QuEST simulator, Cirq returns the outcome of the evaluation as JSON object which we can print directly

In [ ]:
std::cout << result << std::endl;

We can also get a slightly prettier output

In [ ]:
std::cout << result.dump(2) << std::endl;

As this is still not too informative, **LibKet** provides quick access `get<...>()` functions to retrieve information about the ``duration`` of the quantum computation, a ``histogram`` of all measured states, and the ``best`` state, i.e. the one that was measured most often

In [ ]:
std::cout << "duration  : " << cirq.get<QResultType::duration>(result).count() << " seconds" << std::endl;
std::cout << "histogram : " << cirq.get<QResultType::histogram>(result)        << std::endl;
std::cout << "best      : " << cirq.get<QResultType::best>(result)             << std::endl;

Let's increase the number of **shots**, i.e. the number of times we run the quantum circuit in our simulator, to ``20`` to see how the histogram changes 

result = cirq.eval(20);

std::cout << "duration  : " << cirq.get<QResultType::duration>(result).count() << " seconds" << std::endl;
std::cout << "histogram : " << cirq.get<QResultType::histogram>(result)        << std::endl;
std::cout << "best      : " << cirq.get<QResultType::best>(result)             << std::endl;

![QuantumInspire](images/quantuminspire_logo.png)

Let's move on to the next quantum backend supported by **LibKet** - [QuantumInspire](https://quantuminspire.com), which is a multi hardware Quantum Technology platform by QuTech. Without any changes to the quantum expression ``expr_bell``, we create a new Quantum Inspire device, load it with the quantum expression and run it the same way as before. The only difference compared to the Cirq backend is that we need to provide a valid username and password. This can be obtained by creating a free account [here](https://quantuminspire.com/account/create).

In [ ]:
QDevice<QDeviceType::qi_26_simulator, 2> qi("FILL_USERNAME_HERE", "FILL_PASSWORD_HERE");
utils::json result = qi(expr).eval(20);

std::cout << "duration  : " << qi.get<QResultType::duration>(result).count() << " seconds" << std::endl;
std::cout << "histogram : " << qi.get<QResultType::histogram>(result)        << std::endl;
std::cout << "best      : " << qi.get<QResultType::best>(result)             << std::endl;

Let's take a short look under the hood to see how much work is required to write low-level quantum assembly language (QASM) for the two simulators

In [ ]:
std::cout << "------- Cirq -------\n" << cirq    << std::endl;
std::cout << "-- QuantumInspire --\n" << qi  << std::endl;

**LibKet** is designed as backend-agnostic quantum programming framework with a unified API, which means that the core functionality is implemented for all quantum backends. If a particular backend does not support a specific function, e.g., Cirq does not report the ``duration`` of the quantum computation, default values are returned.

Some quantum backends provide extra functionality, which is exposed via **LibKet**. Cirq for instance can export the quantum circuit to [LaTeX](https://www.latex-project.org) code using the [Qcircuit](https://ctan.org/pkg/qcircuit) package

In [ ]:
std::cout << cirq.to_latex() << std::endl;

The resulting circuit looks as follows

![Bell state](images/qiskit_to_latex.png)